In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn
import scipy as sp
import nltk
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math 
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = 200
%matplotlib inline
#import contractions
from bs4 import BeautifulSoup
import unicodedata
import nltk 
nltk.download('stopwords')
from numpy.random import seed
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot
#from matplotlib import pyplot
import pylab
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import norm 
from scipy import stats 
from statsmodels.stats.weightstats import ztest
from matplotlib import rcParams

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nazanin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv(r'C:\Users\Nazanin\Downloads\drugsCom_raw\drugsComTest_raw.csv', sep="\t")
df1 = pd.read_csv(r'C:\Users\Nazanin\Downloads\drugsCom_raw\drugsComTrain_raw.csv', sep="\t")
#display(df, df1)

In [3]:
df2 = df.append(df1)
list(df2)

['Unnamed: 0',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount']

In [4]:
df2.reset_index(drop=True)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed ...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done very well on the Asacol. He has no complaints and shows no side effects. He has taken as many as nine tablets per day at one time. I&#039;ve been v...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for alcohol, smoking, and opioid cessation. People lose weight on it because it also helps control over-eating. I have no doubt that most obesity is caused ...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cycle. After reading some of the reviews on this type and similar birth controls I was a bit apprehensive to start. Im giving this birth control a 9 out ...",9.0,"October 22, 2015",4
5,208087,Zyclara,Keratosis,"""4 days in on first 2 weeks. Using on arms and face. Put vaseline on lips, under eyes and in nostrils to protect from cream. So far no reaction at all. I know I have many pre cancer and thought...",4.0,"July 3, 2014",13
6,215892,Copper,Birth Control,"""I&#039;ve had the copper coil for about 3 months now. I was really excited at the thought of not taking hormones. I&#039;m good with pain however I nearly fainted with insertion, couldn&#039;t be...",6.0,"June 6, 2016",1
7,169852,Amitriptyline,Migraine Prevention,"""This has been great for me. I&#039;ve been on it for 2 weeks and in the last week I only had 3 headaches which went away with 2 Tylenol. I was having chronic daily headaches that wouldn&#039;t go...",9.0,"April 21, 2009",32
8,23295,Methadone,Opiate Withdrawal,"""Ive been on Methadone for over ten years and currently,I am trying to get off of this drug. Ive been decreasing my does 2 mgs per month for over a year. I am at 3 mgs and really starting to feel ...",7.0,"October 18, 2016",21
9,71428,Levora,Birth Control,"""I was on this pill for almost two years. It does work as far as not getting pregnant however my experience at first was it didn&#039;t make a huge difference then 6 or 7 months into it my sex dri...",2.0,"April 16, 2011",3


For hypothesis testing:
+ Now, we are doing hypothesis tests on proportion of birth control, depression, pain and anxiety.
+ We do a comparison (and test) of the mean drug rating between the test and train sets (df1 and df). This item is particularly important because if the conditions are exactly balanced, the people that made the datasets might have stratified over condition. Stratifying guarantees that those classes will be balanced, but makes it more likely that other variables (e.g. review score) not used for stratification will be imbalanced

In [8]:
df2t = len(df2.condition)
df2t

215063

In [27]:
df2.rating[:20]

0     10.0
1      8.0
2      9.0
3      9.0
4      9.0
5      4.0
6      6.0
7      9.0
8      7.0
9      2.0
10     1.0
11     6.0
12     1.0
13    10.0
14     2.0
15     9.0
16    10.0
17     3.0
18     6.0
19    10.0
Name: rating, dtype: float64

In [9]:
cbc = len(df2[df2['condition'] == 'Birth Control'] )
#df2[df2['condition']=='Birth Control'] 

In [10]:
cd = len(df2[df2['condition'] == 'Depression'] )

In [11]:
cp = len(df2[df2['condition'] == 'Pain'] )

In [12]:
ca = len(df2[df2['condition'] == 'Anxiety'] )

In [14]:
print('ratio of Birth Control in the entire data set is: ' + str(cbc/df2t))
print('ratio of Depression in the entire data set is: ' + str(cd/df2t))
print('ratio of Pain in the entire data set is: ' + str(cp/df2t))
print('ratio of Anxiety in the entire data set is: ' + str(ca/df2t))


ratio of Birth Control in the entire data set is: 0.1787197239878547
ratio of Depression in the entire data set is: 0.05656017074066669
ratio of Pain in the entire data set is: 0.03833760339993397
ratio of Anxiety in the entire data set is: 0.036324239873897414


In [5]:
df1t = len(df1.condition)   #df1 is the train
cbc1 = len(df1[df1['condition'] == 'Birth Control'] )
cd1 = len(df1[df1['condition'] == 'Depression'] )
cp1 = len(df1[df1['condition'] == 'Pain'] )
ca1 = len(df1[df1['condition'] == 'Anxiety'] )

print('ratio of Birth Control in the train data set is: ' + str(cbc1/df1t))
print('ratio of Depression in the train data set is: ' + str(cd1/df1t))
print('ratio of Pain in the train data set is: ' + str(cp1/df1t))
print('ratio of Anxiety in the train data set is: ' + str(ca1/df1t))

ratio of Birth Control in the train data set is: 0.1784782110020645
ratio of Depression in the train data set is: 0.05622547226544821
ratio of Pain in the train data set is: 0.03809742276669746
ratio of Anxiety in the train data set is: 0.03660328462401657


In [6]:
dft = len(df.condition)   #df is the test 
cbc0 = len(df[df['condition'] == 'Birth Control'] )
cd0 = len(df[df['condition'] == 'Depression'] )
cp0 = len(df[df['condition'] == 'Pain'])
ca0 = len(df[df['condition'] == 'Anxiety'])

print('ratio of Birth Control in the test data set is: ' + str(cbc0/dft))
print('ratio of Depression in the test data set is: ' + str(cd0/dft))
print('ratio of Pain in the test data set is: ' + str(cp0/dft))
print('ratio of Anxiety in the test data set is: ' + str(ca0/dft))

ratio of Birth Control in the test data set is: 0.17944425845329762
ratio of Depression in the test data set is: 0.0575642599412268
ratio of Pain in the test data set is: 0.039058140832496376
ratio of Anxiety in the test data set is: 0.03548711081352528


In [15]:
delta_bc =  cbc0/dft - cbc1/df1t
print('difference of the rations between Birth Control on the test and train data set  is: ' + str(delta_bc))
delta_d =  cd0/dft - cd1/df1t
print('difference of the rations between Depression on the test and train data set  is: ' + str(delta_d))
delta_p =  cp0/dft - cp1/df1t
print('difference of the rations between Pain on the test and train data set  is: ' + str(delta_p))
delta_a =  ca0/dft - ca1/df1t
print('difference of the rations between Anxiety on the test and train data set  is: ' + str(delta_a))

difference of the rations between Birth Control on the test and train data set  is: 0.0009660474512331041
difference of the rations between Depression on the test and train data set  is: 0.0013387876757785869
difference of the rations between Pain on the test and train data set  is: 0.0009607180657989142
difference of the rations between Anxiety on the test and train data set  is: -0.0011161738104912902


In [14]:
#Hypothesis test:   #df1 is the train set and df is the test set 
# H_0 : mean drug rating for the total and the test data set is the same 
# H_a : mean drug rating for the total and the test data set is not the same 
#Now, let's try frequentist statistical testing. Would you use a one-sample or two-sample test? Why? 
# One Sample test since we are not  comparing the means of the treatment and control groups 
# and it's just one mean value that we are trying to veirfy the Hypothese about. 
# In other words, we are comparing one set of data to a single number and not comparing two sets of data
#In this situation, is it appropriate to use the  𝑡  or  𝑧  statistic?  t b/c the population mean is unknown
# The general rule of thumb for when to use a t score is when your sample: Has a sample size below 30, 
# Has an unknown population standard deviation so we use t statistic. 


In [16]:
np.mean(df2.rating)
print('Mean of drug rating on the entire data set is: ' + str(np.mean(df2.rating)))

np.mean(df1.rating)
print('Mean of drug rating on the train data set is: ' + str(np.mean(df1.rating)))

np.mean(df.rating)
print('Mean of drug rating on the test data set is: ' + str(np.mean(df.rating)))

Mean of drug rating on the entire data set is: 6.990007579174474
Mean of drug rating on the train data set is: 6.994376832799122
Mean of drug rating on the test data set is: 6.97689989956478


In [17]:
np.std(df2.rating)
print('Standard deviation of drug rating on the entire data set is: ' + str(np.std(df2.rating)))

np.std(df1.rating)
print('Standard deviation of drug rating on the train data set is: ' + str(np.std(df1.rating)))

np.std(df.rating)
print('Standard deviation of drug rating on the test data set is: ' + str(np.std(df.rating)))

Standard deviation of drug rating on the entire data set is: 3.2755468822457248
Standard deviation of drug rating on the train data set is: 3.2723190652035488
Standard deviation of drug rating on the test data set is: 3.285176379204773


In [18]:
def permutation_sample(d1, d2):  
    d = np.concatenate((d1, d2))
    perm_d = np.random.permutation(d)
    return perm_d[:len(d1)], perm_d[len(d1):]

In [19]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

In [20]:
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(df1.rating, df.rating)

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(df1.rating, df.rating, diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

# Print the result
print('p-value =', p)

p-value = 0.1402


In [21]:
conf_int_bs_repl = np.percentile(perm_replicates, [0.5, 99.5]) 
print(conf_int_bs_repl)


[-0.04293395  0.04329292]


In [22]:
if p < 0.01: 
    print('We reject H_0 and the true population mean is not 98.6 F.')
else:
        print('Fail to reject H_0')

Fail to reject H_0


### Text pre-processing

+ Removing tags: Removing unnecessary HTML tags, etc. which do not add much value when analyzing text (The BeautifulSoup library does an excellent job in providing necessary functions for this.).

+ Removing accented characters: Removing  accented characters\letters in the text corpus and convert these characters and standardized into ASCII characters, an example is to convert é to e. 

+ Expanding contractions: Converting contraction to its expanded, original form often helps with text standardization, example of which would be, "do not" to "don’t" and "I would" to "I’d".

+ Removing special characters: Removing special characters and symbols often add to the extra noise in unstructured text. More than often, simple regular expressions (regexes) can be used to achieve this.

+ Stemming and lemmatization: The reverse process of inflection is called stemming, which is basically obtaining the base form of a word. Lemmatization is very similar to stemming, the difference being that the root word from lemmatization is always a lexicographically correct word but the root stem may not be so.

+ Removing stopwords: Words with little or no significance in text corpus are known as stopwords, example of which are:  a, an, the. We can use a standard English language stopwords list from nltk and also add our own domain specific stopwords as needed.

+ Other text cleaning to do would be tokenization, removing extra whitespaces, text lower casing as well as spelling corrections, grammatical error corrections, removing repeated characters to name a few. 

In [24]:
# Let's remove special characters, extra whitespaces, digits, stopwords and lower casing the text corpus


#result = re.sub(pattern, repl, string, count=0, flags=0);


#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize, sent_tokenize

wpt = nltk.WordPunctTokenizer()  # $4.99 
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A) #re.I A means treats . as whatever it is. Performs case-insensitive matching. #issues with FLAGS!!!!!
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words] #filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)




In [25]:
norm_corpus = normalize_corpus(df2.review)
norm_corpus

array(['ive tried antidepressants years citalopram fluoxetine amitriptyline none helped depression insomnia amp anxiety doctor suggested changed onto mg mirtazapine medicine saved life thankfully side effects especially common weight gain ive actually lost alot weight still suicidal thoughts mirtazapine saved',
       'son crohns disease done well asacol complaints shows side effects taken many nine tablets per day one time ive happy results reducing bouts diarrhea drastically',
       'quick reduction symptoms', ...,
       'limited improvement months developed bad rash md refused continue medication',
       'ive thyroid medication years spent first synthroid various formulas could stand synthroid never steady every time went dr adjusted could regulate drs played raising lowering mg levels finally person new told see dr beginning beautiful relationship armour thyroid ive never felt better bad symptoms went away feel almost human holds steady longer periods sometimes patient knows bes

##### Bag of Words Model

The bag of words model represents each text document as a numeric vector where each dimension is a specific word from the corpus and the value could be its frequency in the document, occurrence (denoted by 1 or 0) or even weighted values. The model’s name is such because each document is represented literally as a ‘bag’ of its own words, disregarding word orders, sequences and grammar, hence, the Bag of Words model doesn’t consider order of words.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)   #float 
cv_matrix = cv.fit_transform(norm_corpus)   #due to memory error  [20:40]   CONVERT words to number 
#cv_matrix = cv_matrix.toarray()   #visualization purposes 
#cv_matrix

In [ ]:
# get all unique words in the corpus
vocab = cv.get_feature_names()    #shows every single word 
# show document feature vectors                 # number of word occurences in each row of the review column
pd.DataFrame(cv_matrix, columns=vocab)      #won't show due to commenting on the third line of above cell

In the above cell, we can clearly see that each column (dimension) in the feature vectors represents a word from the text corpus and each row represents one of our review rows. The value in any cell, represents the number of times that word (represented by column) occurs in the specific document (represented by row). 
Hence, if a text corpus consists of N unique words across all the documents, we would have an N-dimensional vector for each of the documents.

In [ ]:
 #you can set the n-gram range to 1,2 to get unigrams as well as bigrams

    #In case needed
    
    
#bv = CountVectorizer(ngram_range=(2,2))
#bv_matrix = bv.fit_transform(norm_corpus)

#bv_matrix = bv_matrix.toarray()
#vocab = bv.get_feature_names()
#pd.DataFrame(bv_matrix, columns=vocab)

##### TF-IDF Model

Since the Bag of Words model is based on absolute term frequency, it can cause issues when dealing with large text corpus as there might be some terms which occur frequently across all documents and these may tend to overshadow other terms in the feature set. 
We can overcome this problem by using TF-IDF model by using a scaling or normalizing factor in its computation, which uses a combination of two metrics in its computation, namely: term frequency (tf) and inverse document frequency (idf). TF-IDF stands for Term Frequency-Inverse Document Frequency.

+ tf(w, D) represents the term frequency of the word "w" in document "D", which can be obtained from the Bag of Words model. The term idf(w, D) is the inverse document frequency for the term "w", which can be computed as the log transform of the total number of documents in the corpus "C" divided by the document frequency of the word "w", which is basically the frequency of documents in the corpus where the word "w" occurs. There are multiple variants of this model but they all end up giving quite similar results. Let’s apply this on our corpus now!

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus[:20])
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

##### document similarity in a corpus

Pairwise document similarity in a corpus involves computing document similarity for each pair of documents in a corpus. Thus if you have C documents in a corpus, you would end up with a C x C matrix such that each row and column represents the similarity score for a pair of documents, which represent the indices at the row and column, respectively. There are several similarity and distance metrics that are used to compute document similarity. These include cosine distance/similarity, euclidean distance, manhattan distance, BM25 similarity, jaccard distance and so on. One of the most popular and widely used similarity metric, cosine similarity is used in the following cell and we can compare pairwise document similarity based on their TF-IDF feature vectors.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity   # do not need this one 

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df


#This must indicate these similar documents have some similar features. 
#This is a perfect example of grouping or clustering that can be solved by unsupervised learning especially 
#when dealing with huge corpora of millions of text documents.


+ Pull out subset of most common, most unique words by TF-IDF and point them out
+ Group data by condition and list the 3 most uniquely defining words for each condition
+ pull out the top 10 items by cosine similarity (dropping the diagonal) and remark on them
+ take average rating by word for words that are over a certain threshold of count
+ build a model predicting rating based on words in review